### Download HF dataset from Object Storage

* we're using OCIfs

In [1]:
import os
import oci
from ocifs import OCIFileSystem
import glob
from tqdm import tqdm

# moved namespace to config.py
from config import (
    NAMESPACE,
)

In [2]:
# the name of the HF dataset (the dir where it is saved)
HF_DIR = "atco2_hf_download"

# the bucket where the entire ds will be saved
BUCKET = "atco2_hf"

In [3]:
#
# This code try to get an instance of OCIFileSystem
# first try using Resource Principal, otherwise use api keys
#
try:
    rps = oci.auth.signers.get_resource_principals_signer()

    # if here, we can use rp
    print("Using RP for auth...")

    fs = OCIFileSystem()
except:
    print("Using API Key for auth...")

    default_config = oci.config.from_file()

    # validate the default config file
    oci.config.validate_config(default_config)

    fs = OCIFileSystem(config="~/.oci/config", profile="DEFAULT")

Using API Key for auth...


In [4]:
# traverse all the files and copy in local
for path, subdirs, files in fs.walk(f"oci://{BUCKET}@{NAMESPACE}/"):
    for name in files:
        # get the directory from path
        if "/" not in path:
            remote_dir = ""
        else:
            remote_dir = path.split("/")[-1]

        dest_dir = os.path.join(HF_DIR, remote_dir)

        # if destdir doesn't exist create
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)

        print(f"Copy {os.path.join(dest_dir, name)}...")
        fs.get(os.path.join(path, name), os.path.join(dest_dir, name))

Copy atco2_hf_download/dataset_dict.json...
Copy atco2_hf_download/train/data-00000-of-00001.arrow...
Copy atco2_hf_download/train/dataset_info.json...
Copy atco2_hf_download/train/state.json...
Copy atco2_hf_download/test/data-00000-of-00001.arrow...
Copy atco2_hf_download/test/dataset_info.json...
Copy atco2_hf_download/test/state.json...
